# Stop Detection

<img align="right" src="https://anitagraser.github.io/movingpandas/pics/movingpandas.png">

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/anitagraser/movingpandas/master?filepath=tutorials/3-stop-detection.ipynb)

There are no definitive answers when it comes to detecting / extracting stops from movement trajectories. Due to tracking inaccuracies, movement speed rarely goes to true zero. GPS tracks, for example, tend to keep moving around the object's stop location. 

Suitable stop definitions are also highly application dependent. For example, an application may be interested in analyzing trip purposes. To do so, analysts would be interested in stops that are longer than, for example, 5 minutes and may try to infer the purpose of the stop from the stop location and time. Shorter stops, such as delays at traffic lights, however would not be relevant for this appication. 

In the MovingPandas **TrajectoryStopDetector** implementation, a stop is detected if the movement stays within an area of specified size for at least the specified duration.

## Setup

In [ ]:
%matplotlib inline

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pandas as pd
import geopandas as gpd
from datetime import datetime, timedelta

import sys
sys.path.append("..")
import movingpandas as mpd
print(mpd.__version__)

import warnings
warnings.simplefilter("ignore")

## Loading Geolife Sample

In [ ]:
%%time
df = gpd.read_file('data/demodata_geolife.gpkg')
df['t'] = pd.to_datetime(df['t'])
df = df.set_index('t').tz_localize(None)
print("Finished reading {} rows".format(len(df)))

In [ ]:
traj_collection = mpd.TrajectoryCollection(df, 'trajectory_id')
traj_collection

## Stop Detection with a SingleTrajectory

In [ ]:
my_traj = traj_collection.trajectories[0]
my_traj

In [ ]:
detector = mpd.TrajectoryStopDetector(my_traj)

In [ ]:
traj_plot = my_traj.hvplot(title='Trajectory {}'.format(my_traj.id), line_width=7.0, tiles='CartoLight', color='slategray', frame_width=700, frame_height=700) 
traj_plot

### Stop duration

In [ ]:
%%time
stop_durations = detector.get_stop_time_ranges(min_duration=timedelta(seconds=60), max_diameter=100)

In [ ]:
for x in stop_durations: 
    print(x)

### Stop points

In [ ]:
%%time
stop_points = detector.get_stop_points(min_duration=timedelta(seconds=60), max_diameter=100)

In [ ]:
stop_points['duration_s'] = (stop_points['end_time']-stop_points['start_time']).dt.total_seconds()
stop_points

In [ ]:
stop_point_plot = traj_plot * stop_points.hvplot(geo=True, size='duration_s', color='deeppink')
stop_point_plot

### Stop segments

In [ ]:
%%time
stops = detector.get_stop_segments(min_duration=timedelta(seconds=60), max_diameter=100)

In [ ]:
stops

In [ ]:
stop_segment_plot = stop_point_plot * stops.hvplot( size=200, line_width=7.0, tiles=None, color='orange') 
stop_segment_plot

### Split at stops

In [ ]:
%%time
split = mpd.StopSplitter(my_traj).split(min_duration=timedelta(seconds=60), max_diameter=100)

In [ ]:
split

In [ ]:
stop_segment_plot + split.hvplot(title='Trajectory {} split at stops'.format(my_traj.id), line_width=7.0, tiles='CartoLight', frame_width=700, frame_height=700) 

In [ ]:
for segment in split:
    print(segment)

## Stop Detection for TrajectoryCollections

The process is the same as for individual trajectories.

In [ ]:
%%time
detector = mpd.TrajectoryStopDetector(traj_collection)
stops = detector.get_stop_segments(min_duration=timedelta(seconds=60), max_diameter=100)
len(stops)

In [ ]:
( traj_collection.hvplot(width=700, height=700, line_width=7.0, tiles='CartoLight', color='slategray') * 
 stops.hvplot( size=200, line_width=7.0, tiles=None, color='deeppink') *
 stops.get_start_locations().hvplot(geo=True, size=200, color='deeppink') )

## Continue exploring MovingPandas

1. [Getting started](1-getting-started.ipynb)
1. [Trajectory aggregation (flow maps)](2-generalization-and-aggregation.ipynb)
1. [Stop detection](3-stop-detection.ipynb)